#### 1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

Online Shoppers Purchasing Intention Dataset
https://archive.ics.uci.edu/ml/datasets/Online+Shoppers+Purchasing+Intention+Dataset#

In [3]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
import itertools
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,  mean_squared_error, precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

import matplotlib.pyplot as plt
from time import strptime
%matplotlib inline

In [4]:
df = pd.read_csv('C:/Users/yana/2/online_shoppers_intention.csv')
df.head(5)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [5]:
df.isnull().sum()

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

In [6]:
df.dtypes

Administrative               int64
Administrative_Duration    float64
Informational                int64
Informational_Duration     float64
ProductRelated               int64
ProductRelated_Duration    float64
BounceRates                float64
ExitRates                  float64
PageValues                 float64
SpecialDay                 float64
Month                       object
OperatingSystems             int64
Browser                      int64
Region                       int64
TrafficType                  int64
VisitorType                 object
Weekend                       bool
Revenue                       bool
dtype: object

In [8]:
df.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


#### 2. сделать feature engineering

In [9]:
df['Month'].value_counts()

May     3364
Nov     2998
Mar     1907
Dec     1727
Oct      549
Sep      448
Aug      433
Jul      432
June     288
Feb      184
Name: Month, dtype: int64

In [10]:
df.loc[df['Month']=='June', 'Month'] = 'Jun'
df['Month'] = [(strptime(i,'%b').tm_mon) for i in df['Month']]
df['Weekend'] = np.where(df['Weekend']==False, 0, 1)
df['Revenue'] = np.where(df['Revenue']==False, 0, 1)

In [11]:
cat_features = ['OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend']
num_features = ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration', 'ProductRelated', 
                'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month']

In [13]:
scaler = StandardScaler()
scaler.fit_transform(df[num_features])

array([[-0.69699296, -0.4571914 , -0.3964779 , ..., -0.31717784,
        -0.30882137, -1.66592432],
       [-0.69699296, -0.4571914 , -0.3964779 , ..., -0.31717784,
        -0.30882137, -1.66592432],
       [-0.69699296, -0.4571914 , -0.3964779 , ..., -0.31717784,
        -0.30882137, -1.66592432],
       ...,
       [-0.69699296, -0.4571914 , -0.3964779 , ..., -0.31717784,
        -0.30882137,  0.9868275 ],
       [ 0.50722805, -0.03291592, -0.3964779 , ..., -0.31717784,
        -0.30882137,  0.9868275 ],
       [-0.69699296, -0.4571914 , -0.3964779 , ..., -0.31717784,
        -0.30882137,  0.9868275 ]])

#### 3. обучить любой классификатор (какой вам нравится)

In [14]:
x_data = df.iloc[:,:-1]
y_data = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=7)

In [38]:
cb_model = CatBoostClassifier(silent=True, iterations=90, learning_rate=0.01, depth=5, l2_leaf_reg=4, 
                                auto_class_weights='Balanced', random_state=42, eval_metric='F1',
                               cat_features=cat_features, one_hot_max_size=40, early_stopping_rounds=50)
cb_model.fit(X_train, y_train)

In [39]:
y_predict = cb_model.predict(X_test)

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    return f1, roc, rec, prc

    
f1, roc, rec, prc = evaluate_results(y_test, y_predict)


Classification results:
f1: 68.45%
roc: 85.61%
recall: 81.83%
precision: 58.83%


In [40]:
#сразу создадим таблицу для записи результатов
columns = ['Model', 'F1', 'Roc', 'Recall', 'Precision']
table = pd.DataFrame(columns=columns)
table.loc[len(table)] = ['Standard_model', f1,  roc, rec, prc]

#### 4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

In [41]:
mod_data = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 477/1908 as positives and unlabeling the rest


In [42]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    11853
 1      477
Name: class_test, dtype: int64


In [43]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

#### 5. применить random negative sampling для построения классификатора в новых условиях

In [44]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(477, 19) (477, 19)


In [45]:
cb_model.fit(sample_train.iloc[:,:-2], sample_train.iloc[:,-2])
y_predict = cb_model.predict(sample_test.iloc[:,:-2])

In [46]:
f1, roc, rec, prc = evaluate_results(sample_test.iloc[:,-2], y_predict)
table.loc[len(table)] = ['PU_model', f1,  roc, rec, prc]

Classification results:
f1: 61.29%
roc: 84.60%
recall: 80.54%
precision: 49.46%


#### 6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

In [47]:
table

,Model,F1,Roc,Recall,Precision
0,Standard_model,0.684515,0.856142,0.818339,0.588308
1,PU_model,0.612876,0.846031,0.805374,0.494648


#### 7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [48]:
for i in np.linspace(0.1, 0.8, 15):
    pos_sample_len = int(np.ceil(i * len(pos_ind)))
    pos_sample = pos_ind[:pos_sample_len]
    mod_data['class_test'] = -1
    mod_data.loc[pos_sample,'class_test'] = 1
    x_data = mod_data.iloc[:,:-2].values 
    y_labeled = mod_data.iloc[:,-1].values 
    y_positive = mod_data.iloc[:,-2].values
    mod_data = mod_data.sample(frac=1)
    neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
    sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
    pos_sample = mod_data[mod_data['class_test']==1]
    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
    cb_model.fit(sample_train.iloc[:,:-2], sample_train.iloc[:,-2])
    y_predict = cb_model.predict(sample_test.iloc[:,:-2])
    f1, roc, rec, prc = evaluate_results(sample_test.iloc[:,-2], y_predict)
    table.loc[len(table)] = [f'PU_model {round(i, 2)}', f1,  roc, rec, prc]

Classification results:
f1: 64.74%
roc: 84.33%
recall: 79.55%
precision: 54.58%
Classification results:
f1: 63.79%
roc: 84.52%
recall: 79.97%
precision: 53.06%
Classification results:
f1: 62.31%
roc: 84.39%
recall: 80.09%
precision: 50.99%
Classification results:
f1: 61.57%
roc: 84.48%
recall: 79.93%
precision: 50.07%
Classification results:
f1: 60.18%
roc: 84.39%
recall: 79.67%
precision: 48.35%
Classification results:
f1: 59.12%
roc: 84.91%
recall: 80.89%
precision: 46.58%
Classification results:
f1: 56.68%
roc: 84.40%
recall: 79.85%
precision: 43.94%
Classification results:
f1: 54.97%
roc: 84.08%
recall: 79.22%
precision: 42.09%
Classification results:
f1: 53.05%
roc: 84.60%
recall: 80.55%
precision: 39.55%
Classification results:
f1: 51.43%
roc: 84.17%
recall: 79.03%
precision: 38.11%
Classification results:
f1: 48.74%
roc: 84.84%
recall: 80.62%
precision: 34.92%
Classification results:
f1: 45.30%
roc: 84.67%
recall: 80.41%
precision: 31.53%
Classification results:
f1: 41.06%
roc: 

In [49]:
table

,Model,F1,Roc,Recall,Precision
0,Standard_model,0.684515,0.856142,0.818339,0.588308
1,PU_model,0.612876,0.846031,0.805374,0.494648
2,PU_model 0.1,0.647371,0.843318,0.795495,0.545750
3,PU_model 0.15,0.637931,0.845223,0.799746,0.530578
4,PU_model 0.2,0.623097,0.843898,0.800945,0.509880
5,PU_model 0.25,0.615686,0.844842,0.799273,0.500683
6,PU_model 0.3,0.601786,0.843882,0.796690,0.483501
7,PU_model 0.35,0.591207,0.849102,0.808874,0.465848
8,PU_model 0.4,0.566823,0.843984,0.798486,0.439355
9,PU_model 0.45,0.549716,0.840843,0.792221,0.420881


В данном случае результат обычной классификации лучше, чем результаты PU моделей при разном соотношении P и U. В модели PU чем меньше кол-во Р, тем лучше результаты. Видимо, потому что данные не очень похожи друг на друга, при большом кол-ве Р значительно увеличивается кол-во ложнопозитивных результатов, precision заметно снижается. Поскольку датасет несбалансированный, в catboost поставлен параметр "auto_class_weights='Balanced'". Если его не ставить, при обычной классификации precision будет выше (recall и f1 ниже), а для PU модели тренд остается тем же: наилучшие результаты при минимальном значении P и очень низкий precision при максимальном.